# SEQUENCE CLASSIFICATION

In [1]:
from __future__ import print_function
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader

LOAD DATASET

In [19]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5), (0.5))])

root = './data'
train_data = datasets.MNIST(root, train=True, download=True, transform=transform)
valid_data = datasets.MNIST(root, train=False, download=True)

batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True,
                          num_workers=4, drop_last=False)
valid_loader = DataLoader(valid_data, batch_size=100, shuffle=False,
                          num_workers=4)

# DEFINE MODEL

In [42]:
class MNISTRNN(nn.Module):
    def __init__(self, input_size, n_hidden, output_size):
        super(MNISTRNN, self).__init__()
        
        self.rnn = nn.RNN(input_size=input_size, hidden_size=n_hidden, num_layers=1)
        self.W1 = nn.Parameter(torch.randn([input_size, n_hidden]).type(torch.float))
        self.b1 = nn.Parameter(torch.randn([n_hidden]).type(torch.float))
        
        self.W = nn.Parameter(torch.randn([n_hidden, output_size]).type(torch.float))
        self.b = nn.Parameter(torch.randn([output_size]).type(torch.float))
        
    def forward(self, hidden, X):
        X = X.transpose(0, 1)
        X = X.reshape(-1, 28, 28)
#         H = torch.mm(X, self.W1) + self.b1
        print(X.shape, hidden.shape)
        outputs, hidden = self.rnn(X, hidden)
        outputs = outputs[-1]
        outputs = torch.mm(outputs, self.W) + self.b
        return outputs

# DEFINE PARAMETERS

In [43]:
diminput = 28
dimhidden = 128
dimoutput = 10
nsteps = 28
learning_rate = 0.001
epochs = 5

model = MNISTRNN(input_size=diminput, n_hidden=dimhidden, output_size=dimoutput)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [44]:
model.train()
for epoch in range(epochs):
    avg_loss = 0.
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.view((batch_size, nsteps, diminput))
        hidden = torch.zeros(1, batch_size, dimhidden, requires_grad=True)
        output = model(hidden, inputs)
        loss = criterion(output, targets)
        avg_loss += losses
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    avg_loss /= len(train_loader)
    print('Epoch:', '%04d' % (epoch), 'Loss =', '{:.6f}'.format(avg_loss))
        
print ('OPTIMIZATION FINISHED')

torch.Size([64, 28, 28]) torch.Size([1, 64, 128])


RuntimeError: Expected hidden size (1, 28, 128), got [1, 64, 128]